In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2


In [ ]:
from datasets import load_dataset

# Load TinyStories using streaming mode (no caching)
dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)

# Convert first few items into a list
from itertools import islice
sample = list(islice(dataset, 5))

# Display sample
for i, story in enumerate(sample):
    print(f"\nStory {i+1}:")
    print(story)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]


Story 1:
{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}

Story 2:
{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tr

In [ ]:
import os
os.environ["HF_DATASETS_CACHE"] = "/content/hf_cache"

from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories")

train_dataset = dataset["train"]
print(train_dataset[0])


(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}


In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load tokenizer and dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # avoid pad token issues

dataset = load_dataset("roneneldan/TinyStories", split="train")

# Token limit
MAX_TOKENS = 1_000_000
total_tokens = 0
texts = []

for example in dataset:
    text = example.get("text", "").strip()
    if not text:
        continue  # skip empty lines
    tokens = tokenizer.encode(text, add_special_tokens=False)
    if len(tokens) == 0:
        continue  # skip texts that tokenize to nothing
    if total_tokens + len(tokens) > MAX_TOKENS:
        break
    texts.append(text)
    total_tokens += len(tokens)

print(f"Loaded {len(texts)} stories totaling approximately {total_tokens} tokens.")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors


Loaded 4861 stories totaling approximately 999816 tokens.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2Tokenizer
from datasets import load_dataset

# ------------------- Config -------------------
class Config:
    vocab_size = 50257
    block_size = 128
    n_layers = 4
    n_heads = 4
    d_model = 256
    d_ff = 1024
    dropout = 0.1
    batch_size = 32
    lr = 3e-4
    max_tokens = 1_000_000

config = Config()

# ------------------- Positional Encoding -------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

# ------------------- Multi-Head Self-Attention -------------------
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0
        self.d_head = d_model // n_heads
        self.n_heads = n_heads
        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        self.o_proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv_proj(x).view(B, T, self.n_heads, 3 * self.d_head)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / (self.d_head ** 0.5)
        att = att.masked_fill(torch.tril(torch.ones(T, T, device=x.device)) == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)

        out = att @ v
        out = out.transpose(1, 2).contiguous().view(B, T, C)
        return self.o_proj(out)

# ------------------- Feed-Forward Network -------------------
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(config.dropout)
        )

    def forward(self, x):
        return self.net(x)

# ------------------- Transformer Block -------------------
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x



In [ ]:
# ------------------- GPT Model -------------------
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_encoding = PositionalEncoding(config.d_model, config.block_size)
        self.blocks = nn.Sequential(*[
            TransformerBlock(config.d_model, config.n_heads, config.d_ff)
            for _ in range(config.n_layers)
        ])
        self.ln_f = nn.LayerNorm(config.d_model)
        self.head = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(self, x):
        x = self.token_embedding(x)
        x = self.position_encoding(x)
        x = self.blocks(x)
        x = self.ln_f(x)
        return self.head(x)

# ------------------- Tokenization & Data Prep -------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
raw_dataset = load_dataset("roneneldan/TinyStories", split="train")

texts = []
total_tokens = 0
for example in raw_dataset:
    text = example.get("text", "").strip()
    if not text:
        continue
    tokens = tokenizer.encode(text, add_special_tokens=False, truncation=True, max_length=config.block_size)
    if len(tokens) == 0:
        continue
    if total_tokens + len(tokens) > config.max_tokens:
        break
    texts.append(text)
    total_tokens += len(tokens)

all_tokens = []
for text in texts:
    token_ids = tokenizer.encode(text, add_special_tokens=False, truncation=True, max_length=config.block_size)
    all_tokens.extend(token_ids)

all_tokens = all_tokens[: (len(all_tokens) // config.block_size) * config.block_size]
input_ids = torch.tensor(all_tokens).view(-1, config.block_size)
targets = input_ids.clone()
dataset = TensorDataset(input_ids, targets)
dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)




In [ ]:
import os
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPTModel(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss()

start_epoch = 0
num_epochs = 3
save_every = 1
checkpoint_path = "gpt_checkpoint.pth"

# --- Resume training if checkpoint exists ---
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    print(f"Resuming training from epoch {start_epoch}")

model.train()
for epoch in range(start_epoch, num_epochs):
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Get predicted tokens (batch_size, seq_len)
        preds = torch.argmax(logits, dim=-1)

        # Flatten and store predictions and labels for metrics calculation
        all_preds.extend(preds.view(-1).cpu().numpy())
        all_labels.extend(y.view(-1).cpu().numpy())

        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch {epoch + 1} Batch {batch_idx + 1}/{len(dataloader)} | Batch Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(dataloader)

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    print(f"Epoch {epoch + 1} completed | Avg Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | F1 Score: {f1:.4f}")

    # Save checkpoint every 'save_every' epochs
    if (epoch + 1) % save_every == 0 or (epoch + 1) == num_epochs:
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }, checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch + 1}")

print("Training complete!")


Epoch 1 Batch 10/245 | Batch Loss: 8.0642
Epoch 1 Batch 20/245 | Batch Loss: 6.2387
Epoch 1 Batch 30/245 | Batch Loss: 4.8280
Epoch 1 Batch 40/245 | Batch Loss: 3.9184
Epoch 1 Batch 50/245 | Batch Loss: 3.1076
Epoch 1 Batch 60/245 | Batch Loss: 2.5697
Epoch 1 Batch 70/245 | Batch Loss: 2.2351
Epoch 1 Batch 80/245 | Batch Loss: 1.9616
Epoch 1 Batch 90/245 | Batch Loss: 1.7506
Epoch 1 Batch 100/245 | Batch Loss: 1.6238
Epoch 1 Batch 110/245 | Batch Loss: 1.4087
Epoch 1 Batch 120/245 | Batch Loss: 1.2203
Epoch 1 Batch 130/245 | Batch Loss: 1.1535
Epoch 1 Batch 140/245 | Batch Loss: 1.0915
Epoch 1 Batch 150/245 | Batch Loss: 1.0730
Epoch 1 Batch 160/245 | Batch Loss: 0.9002
Epoch 1 Batch 170/245 | Batch Loss: 0.9036
Epoch 1 Batch 180/245 | Batch Loss: 0.8734
Epoch 1 Batch 190/245 | Batch Loss: 0.7776
Epoch 1 Batch 200/245 | Batch Loss: 0.7457
Epoch 1 Batch 210/245 | Batch Loss: 0.7112
Epoch 1 Batch 220/245 | Batch Loss: 0.6224
Epoch 1 Batch 230/245 | Batch Loss: 0.6169
Epoch 1 Batch 240/24

In [ ]:
# ------------------- Text Generation -------------------
def generate_text(prompt, model, tokenizer, max_new_tokens=50):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_ids)
            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            input_ids = torch.cat((input_ids, next_token), dim=1)
    return tokenizer.decode(input_ids[0])

# Example usage
print(generate_text("Once upon a time", model, tokenizer))

Once upon a time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time time
